강의_6기_AI응용_6차시_01_GramMatrix와 Loss.ipynb

In [1]:
# PyTorch의 핵심 라이브러리를 불러옴.
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision
from torchvision import transforms

from PIL import Image
from collections import OrderedDict

In [2]:
# Gram Matrix 계산하는 파이토치 모듈 정의
class GramMatrix(nn.Module):
    def forward(self, input):
        b, c, h, w = input.size()
        features = input.view(b, c, h * w)

        # 배치 행렬곱(bmm) -> gram matrix 계산
        # features: (b, c, h*w)
        # features.transpose(1,2): (b, h*w, c)
        # 출력 : (b, c, c) 
        G = torch.bmm(features, features.transpose(1,2))

        # G 행렬을 h*w로 나누어 정규화
        G.div_(h * w)

        return G


In [3]:
# gram matrix에 대해 MSE(평균제곱오차) 손실 계산하는 모듈
class GramMSELoss(nn.Module):
    def forward(self, input, target):
        # GramMatrix 모듈 통과 > gram matrix 생성
        # target은 이미지의 gram matrix
        out = nn.MSELoss()(GramMatrix()(input), target)

        return out


[VGG 구조 패턴]

- Block 1: 3→64→64 (얕은 특징: 선, 모서리)
- Block 2: 64→128→128 (중간 특징: 질감)
- Block 3: 128→256→256→256 (깊은 특징: 패턴)
- Block 4: 256→512→512→512 (더 복잡한 특징)
- Block 5: 512→512→512→512 (추상적 특징)

In [4]:
class VGG(nn.Module):
    def __init__(self, pool='max'):
        super(VGG, self).__init__()
        #vgg modules
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        if pool == 'max': # 특징 최대값 추출(뚜렷한 특징)
            self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)
        elif pool == 'avg': # 특징을 평균 추출(부드러운 특징)
            self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool3 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool4 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool5 = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x, out_keys):
        out = {}
        out['r11'] = F.relu(self.conv1_1(x))
        out['r12'] = F.relu(self.conv1_2(out['r11']))
        out['p1'] = self.pool1(out['r12'])
        out['r21'] = F.relu(self.conv2_1(out['p1']))
        out['r22'] = F.relu(self.conv2_2(out['r21']))
        out['p2'] = self.pool2(out['r22'])
        out['r31'] = F.relu(self.conv3_1(out['p2']))
        out['r32'] = F.relu(self.conv3_2(out['r31']))
        out['r33'] = F.relu(self.conv3_3(out['r32']))
        out['r34'] = F.relu(self.conv3_4(out['r33']))
        out['p3'] = self.pool3(out['r34'])
        out['r41'] = F.relu(self.conv4_1(out['p3']))
        out['r42'] = F.relu(self.conv4_2(out['r41']))
        out['r43'] = F.relu(self.conv4_3(out['r42']))
        out['r44'] = F.relu(self.conv4_4(out['r43']))
        out['p4'] = self.pool4(out['r44'])
        out['r51'] = F.relu(self.conv5_1(out['p4']))
        out['r52'] = F.relu(self.conv5_2(out['r51']))
        out['r53'] = F.relu(self.conv5_3(out['r52']))
        out['r54'] = F.relu(self.conv5_4(out['r53']))
        out['p5'] = self.pool5(out['r54'])
        return [out[key] for key in out_keys]

In [6]:
vgg = VGG()
criterion = nn.MSELoss()

# 로드할 이미지 파일 이름들을 정의함. ('vangogh_starry_night.jpg', 'Tuebingen_Neckarfront.jpg')
img1 = "vangogh_starry_night.jpg"
img2 = "Tuebingen_Neckarfront.jpg"

# 이미지 디렉토리와 파일 이름을 결합하여 PIL Image 객체 리스트로 로드함.
img1 = Image.open(img1)
img2 = Image.open(img2)
imgs = []
imgs.append(img1)
imgs.append(img2)

img_size = 512

prep = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(), # c h w [0-1]
    transforms.Lambda(lambda x: x[torch.LongTensor([2,1,0])]), # rgb -> bgr
    transforms.Normalize(
        mean=[0.40760392, 0.45795686, 0.48501961],
        std=[1,1,1]
        ),
    transforms.Lambda(lambda x: x.mul_(255)), # 스케일
])

# pil 객체에 사전 정의된 전처리 함수 적용
img_torch = [prep(img) for img in imgs]

# variable 없어도 되는 것.
if torch.cuda.is_available():
    # 각 텐서에 배치 차원(unsqueeze(0))을 추가하고 GPU로 이동시킨 후, Variable로 감싸서 저장함.
    # imgs_torch = [Variable(img.unsqueeze(0).cuda()) for img in img_torch]
    imgs_torch = [img.unsqueeze(0).cuda() for img in img_torch]
else:
    # 각 텐서에 배치 차원만 추가하고 Variable로 감싸서 저장함.
    # imgs_torch = [Variable(img.unsqueeze(0)) for img in img_torch]
    imgs_torch = [img.unsqueeze(0) for img in img_torch]

style_img, content_img = imgs_torch
# opt_img = Variable(content_img.data.clone(), requires_grad=True)
opt_img = content_img.data.clone().detach().requires_grad_(True)
# 신경망 스타일 전이에서 CNN 가중치 학습X.
# opt_img의 픽셀값만 학습하며 업데이트

# 스타일 손실 계산할 VGG 레이어 이름정의
# VGG 내부 특정 합성곱 레이어
style_layers = ['r11','r21','r31','r41', 'r51']

# 콘텐츠 손실 계산할 VGG 레이어 이름정의
# 일반적으로 중간정도 레이어 중 하나 사용
content_layers = ['r42']

loss_layers = style_layers + content_layers

# 각 스타일에 대해 GramMSELoss 모듈을 사용하고, 
# 콘텐츠에 대해 MSELoss 모듈을 사용하도록 리스트를 정의함.
loss_fns = [GramMSELoss()] * len(style_layers) + [nn.MSELoss()] * len(content_layers)

# GPU (CUDA) 사용이 가능하다면, 모든 손실 함수 모듈을 GPU 메모리로 이동시킴.
if torch.cuda.is_available():
    # loss_fns 리스트의 요소들을 새로운 모듈 인스턴스로 만들고 .cuda()를 적용
    # 리스트 복사를 방지하고 정확하게 GPU로 이동하기 위해 수정
    # for _ in style_layers : 개수만큼 인스턴스화
    loss_fns = [GramMSELoss().to(device) for _ in style_layers] + \
               [nn.MSELoss().to(device) for _ in content_layers]
else:
    loss_fns = [GramMSELoss().to(device) for _ in style_layers] + \
               [nn.MSELoss().to(device) for _ in content_layers]

# 스타일 손실에 적용할 가중치(알파)
# 깊은 레이어(복잡한 패턴을 추출하는 레이어)일수록 낮은 가중치를 주는 경향
# 왜냐하면, 가중치가 감소되니까.
style_weights = [1e3/n**2 for n in [64, 128, 256, 512, 512]]

# 콘텐츠 손실에 적용할 가중치(베타)
content_weights = [1e0]
content_weights = [1] * len(content_layers)

# 최적화할 목표값 (style target) 계산
# style_img를 VGG를 통과시켜서 각 레이어의 출력값을 계산
# 변화 추적X
# [A_r11, A_r21, ...] A.shape [b, c, h, w]
# GramMatrix(A).shape [b, c, c]
# detach() : 계산 그래프 분리(역전파 시 gradient 계산X)
style_targets = [GramMatrix()(A).detach() for A in vgg(style_img, style_layers)]

# 최적화 목표값(content targets)을 계산함.
# 콘텐츠 이미지(content_image)를 VGG에 통과시켜 콘텐츠 레이어의 특징 맵을 추출하고 변화도 추적에서 제외(detach)했음.
# content_image 또한 이미 .cuda() 또는 .to(device)로 GPU에 로드되어 있다고 가정합니다.
content_targets = [A.detach() for A in vgg(content_image, content_layers)]

# 최종적으로 사용할 모든 목표값 리스트를 정의함.
targets = style_targets + content_targets

NameError: name 'device' is not defined

In [ ]:
targets